In [1]:
import torch
import numpy as np

In [2]:
def rescale(vec):
    min_val = vec.min()
    max_val = vec.max()
    if min_val == max_val:
        return torch.zeros_like(vec)
    else:
        return (vec - min_val) / (max_val - min_val)

In [80]:
def sparse_vector_field(coords, vecs, x_dim=100, y_dim=100):
    x, y = coords.t()
    x = rescale(x) * (x_dim - 1)
    y = rescale(y) * (y_dim - 1)

    indices = torch.stack([x.round().long(), y.round().long()]).t()
    _, count_indices, counts = np.unique(indices, axis=0, return_inverse=True, return_counts=True)
    counts = torch.tensor(counts[count_indices]).unsqueeze(1)

    return torch.sparse.FloatTensor(
        indices.t(),
        vecs / counts,
        torch.Size([x_dim, y_dim, 2])
    ).coalesce()

In [183]:
def sparse_vector_field_2(coords, vecs, x_dim=100, y_dim=100):
    x, y = coords.t()
    x = rescale(x) * (x_dim - 1)
    y = rescale(y) * (y_dim - 1)

    indices = torch.stack([x.round().long(), y.round().long()]).t()
    _, count_indices, counts = np.unique(indices, axis=0, return_inverse=True, return_counts=True)
    counts = torch.tensor(counts[count_indices]).unsqueeze(1)

    field = torch.zeros((x_dim, y_dim, 2))
    for idx, ct, v in zip(indices, counts, vecs):
        field[tuple(idx)] += v / ct

    return field

In [187]:
v = 10*torch.rand((10,2))
field = sparse_vector_field(v, v, x_dim=3, y_dim=3)
field2 = sparse_vector_field_2(v, v, x_dim=3, y_dim=3)

In [189]:
field.to_dense()

tensor([[[1.2872, 1.4453],
         [1.2417, 5.4896],
         [0.0000, 0.0000]],

        [[4.5084, 2.1133],
         [5.4112, 4.3041],
         [5.7032, 8.2407]],

        [[8.1804, 1.0203],
         [0.0000, 0.0000],
         [0.0000, 0.0000]]])

In [190]:
field2

tensor([[[1.2872, 1.4453],
         [1.2417, 5.4896],
         [0.0000, 0.0000]],

        [[4.5084, 2.1133],
         [5.4112, 4.3041],
         [5.7032, 8.2407]],

        [[8.1804, 1.0203],
         [0.0000, 0.0000],
         [0.0000, 0.0000]]])

In [263]:
def divergence(coords, vec, x_dim=100,y_dim=100):
    vf = sparse_vector_field_2(coords, vec, x_dim=x_dim, y_dim=y_dim)#.to_dense()

    x1 = vf.transpose(0,1)[:,:-1,0]
    x2 = vf.transpose(0,1)[:,1:,0]
    y1 = vf[:,:-1,1]
    y2 = vf[:,1:,1]

    x_div = (x2-x1)[(x1*x2).bool()]
    if x_div.shape[0] == 0:
        raise Exception('Not enough samples in x-direction')
    x_div = x_div.pow(2).sum() / x_div.shape[0] * (x_dim - 1)
    y_div = (y2-y1)[(y1*y2).bool()]
    if y_div.shape[0] == 0:
        raise Exception('Not enough samples in y-direction')
    y_div = y_div.pow(2).sum() / y_div.shape[0] * (y_dim - 1)

    return (x_div + y_div)

In [264]:
v = torch.rand((500,2), requires_grad=True)
c = v.clone().detach()
baz = sparse_vector_field_2(c, c, x_dim=10, y_dim=10)
div = divergence(c, 10*v, x_dim=10,y_dim=10)
div.backward()
v.grad

tensor([[-0.3580,  0.3258],
        [ 0.0633,  0.0053],
        [ 0.0525,  0.0115],
        [ 0.5262,  0.0119],
        [ 0.7998,  0.9830],
        [ 0.0371, -0.4548],
        [-0.0138,  0.1014],
        [ 0.1552,  0.4071],
        [ 0.0196,  0.0831],
        [ 0.1507,  0.3797],
        [ 0.1552,  0.4071],
        [ 0.0542, -0.0131],
        [-0.0986, -0.0534],
        [-0.0138,  0.1014],
        [-0.1351,  0.0357],
        [-0.1506, -0.0704],
        [ 0.0833, -0.1153],
        [ 0.0525,  0.0115],
        [-0.3623,  0.3358],
        [-0.1892,  0.3350],
        [-0.0994,  0.0647],
        [-0.1895, -0.0645],
        [ 0.0371, -0.4548],
        [-0.7987, -0.5644],
        [-0.1586,  0.0593],
        [ 0.9295, -1.1404],
        [-0.1164,  0.1102],
        [-0.0715,  0.0042],
        [ 0.0196,  0.0831],
        [-0.0994,  0.0647],
        [ 0.1449, -0.4306],
        [ 1.8973, -1.1508],
        [ 0.0525,  0.0115],
        [-0.8601, -0.8516],
        [-0.0715,  0.0042],
        [-0.0715,  0

In [261]:
div

tensor(18.9113, grad_fn=<AddBackward0>)

In [262]:
baz

tensor([[[0.0269, 0.0287],
         [0.0183, 0.1081],
         [0.0062, 0.2637],
         [0.0294, 0.3504],
         [0.0506, 0.4248],
         [0.0258, 0.5440],
         [0.0252, 0.6679],
         [0.0382, 0.7962],
         [0.0000, 0.0000],
         [0.0382, 0.9638]],

        [[0.1062, 0.0344],
         [0.1122, 0.1119],
         [0.0967, 0.2186],
         [0.1233, 0.3391],
         [0.0941, 0.4740],
         [0.1066, 0.5647],
         [0.1218, 0.6577],
         [0.1117, 0.7824],
         [0.1377, 0.8721],
         [0.1139, 0.9722]],

        [[0.2200, 0.0350],
         [0.2218, 0.1068],
         [0.2199, 0.2014],
         [0.2216, 0.3247],
         [0.2310, 0.4359],
         [0.2238, 0.5421],
         [0.2131, 0.6795],
         [0.2050, 0.7851],
         [0.1698, 0.9016],
         [0.1809, 0.9833]],

        [[0.3314, 0.0303],
         [0.3282, 0.1276],
         [0.3371, 0.2249],
         [0.3483, 0.3362],
         [0.3720, 0.4585],
         [0.3301, 0.5779],
         [0.3457, 0.66

In [233]:
foo = torch.tensor([1.,2.,3.], requires_grad=True)
bar = (5*foo).sum()
bar.backward()
foo.grad

tensor([5., 5., 5.])